In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 读取A和B两个CSV文件
A = pd.read_csv('MFFmatrix-onlyS.csv')
B = pd.read_csv('Rdkitmatrix+sym descriptor-only S.csv')

# 计算A数据的相关性系数矩阵
A_correlation_matrix = A.corr()

# 打印A的相关性系数矩阵
print("Correlation Matrix of A:\n", A_correlation_matrix)

# 找到相关性系数为1的列名（排除自相关）
to_remove = set()
for col in A_correlation_matrix.columns:
    for idx in A_correlation_matrix.index:
        if col != idx and A_correlation_matrix.loc[idx, col] == 1:
            if col != idx:  # 如果变量名字不同
                to_remove.add(col)
                break  # 找到一个就跳出循环

# 打印要删除的变量名字
print("Variables to remove:\n", to_remove)

# 删除相关性系数为1的列
A1 = A.drop(columns=to_remove)

# 打印更新后的A数据
print("Updated A Data:\n", A1)

# 保存新的A1 CSV文件
A1.to_csv('A1.csv', index=False)
print("Updated A1 CSV saved to 'A1.csv'")

# 读取A1 CSV文件
A1 = pd.read_csv('A1.csv')

# 初始化标准化器
scaler = StandardScaler()

# 标准化A1数据
A1_scaled = scaler.fit_transform(A1)
A1_scaled = pd.DataFrame(A1_scaled, columns=A1.columns)
A1_scaled.drop(columns=A1_scaled.std()[A1_scaled.std() == 0].index, inplace=True)

# 标准化B数据
B_scaled = scaler.fit_transform(B)
B_scaled = pd.DataFrame(B_scaled, columns=B.columns)
B_scaled.drop(columns=B_scaled.std()[B_scaled.std() == 0].index, inplace=True)

# 确保数据中没有缺失值
A1.dropna(inplace=True)
B_scaled.dropna(inplace=True)

# 初始化一个空的DataFrame来存储相关性系数
correlation_matrix = pd.DataFrame(index=A1.columns, columns=B_scaled.columns)

# 计算相关性系数
for a_col in A1.columns:
    for b_col in B_scaled.columns:
        correlation = A1[a_col].corr(B_scaled[b_col])
        correlation_matrix.at[a_col, b_col] = correlation

# 将相关性矩阵转换为浮点数类型
correlation_matrix = correlation_matrix.astype(float)
# 获取矩阵的第一列的行名和第一行的列名
first_column_row_names = correlation_matrix.index
first_row_column_names = correlation_matrix.columns

# 找到重复的列名
columns_to_drop = [col_name for col_name in first_row_column_names if col_name in first_column_row_names]

# 删除重复列名所在的列
correlation_matrix = correlation_matrix.drop(columns=columns_to_drop)

# 删除某一行或某一列全是NaN的行和列
correlation_matrix.dropna(axis=0, how='all', inplace=True)
correlation_matrix.dropna(axis=1, how='all', inplace=True)
# 打印更新后的相关性矩阵
print("Updated Correlation Matrix:\n", correlation_matrix)
# 绘制热图
plt.figure(figsize=(150, 150))  # 调整图像大小以适应较小的矩阵
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False,center=0, annot_kws={"size": 30}, cbar=True)
plt.title('Updated Correlation Matrix Heatmap between MFF and Physicochemical properties features', fontsize=40)
plt.xlabel('Physicochemical properties', fontsize=80)
plt.ylabel('MFF', fontsize=80)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=80) # 设置x轴字体大小
ax.set_yticklabels(ax.get_yticklabels(), fontsize=80) # 设置y轴字体大小
plt.show()
print(correlation_matrix.shape)
# 将更新后的相关性矩阵保存为CSV文件
correlation_matrix.to_csv('updated_correlation_matrix_between_Original_and_Encoded_Features.csv')
print("Updated correlation matrix saved to 'updated_correlation_matrix_between_Original_and_Encoded_Features.csv'")



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# 读取CSV文件
file_path = 'updated_correlation_matrix_between_Original_and_Encoded_Features.csv'
data = pd.read_csv(file_path, index_col=0)
plt.figure(figsize=(200, 200))  # 调整图像大小以适应较小的矩阵
ax=sns.heatmap(data, cmap='coolwarm', annot=False,center=0, annot_kws={"size": 100}, cbar=True)
plt.title('Updated Correlation Matrix Heatmap between MFF and Physicochemical properties features', fontsize=40)
plt.xlabel('Physicochemical properties', fontsize=40)
plt.ylabel('MFF', fontsize=40)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=40) # 设置x轴字体大小
ax.set_yticklabels(ax.get_yticklabels(), fontsize=40) # 设置y轴字体大小
# 设置颜色条字体大小
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=60)  # 调整颜色条刻度字体大小
#cbar.set_label('Correlation Value', fontsize=60)  # 设置颜色条标签及其字体大小
plt.show()

In [ ]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import Alignment

# 读取相关性矩阵CSV文件
correlation_matrix_path = 'updated_correlation_matrix_between_Original_and_Encoded_Features.csv'
updated_correlation_matrix = pd.read_csv(correlation_matrix_path, index_col=0)

def correlate_features(feature_to_search):
    threshold_similar = 0.7
    threshold_explain = 0.4

    similar_features = []
    explain_features = []

    if feature_to_search in reordered_RDkit.columns:
        Similar = reordered_RDkit_corr.loc[feature_to_search, :]
        for rdkit in reordered_RDkit.columns:
            if rdkit != feature_to_search and abs(Similar[rdkit]) > threshold_similar:
                similar_features.append((rdkit, Similar[rdkit]))
        similar_features = sorted(similar_features, key=lambda x: abs(x[1]), reverse=True)
        
        # 如果没有找到相似特征，填入 'None'
        if not similar_features:
            similar_features.append(('None', 'None'))

        if feature_to_search in updated_correlation_matrix.index:
            representation = updated_correlation_matrix.loc[feature_to_search, :]
            for mff in updated_correlation_matrix.columns:
                if pd.notna(representation[mff]) and abs(representation[mff]) > threshold_explain:
                    explain_features.append((mff, representation[mff]))

        if feature_to_search in updated_correlation_matrix.columns:
            representation = updated_correlation_matrix.loc[:, feature_to_search]
            for mff in updated_correlation_matrix.index:
                if pd.notna(representation[mff]) and abs(representation[mff]) > threshold_explain:
                    explain_features.append((mff, representation[mff]))

        explain_features = list(set(explain_features))
        explain_features = sorted(explain_features, key=lambda x: abs(x[1]), reverse=True)
        
        # 如果没有找到解释特征，填入 'None'
        if not explain_features:
            explain_features.append(('None', 'None'))

    elif feature_to_search in updated_correlation_matrix.index:
        explain_features = ['MFF descriptor itself']

    return similar_features, explain_features


In [ ]:
# 读取 feature_to_search 列表的 CSV 文件路径
features_to_search_path = 'S体系/20240611/S-体系-all metrics/shap_results-n-PrOH/MLP-top-features.csv'
features_to_search_df = pd.read_csv(features_to_search_path, encoding='utf-8')
# 读取 product 信息的 CSV 文件路径
product_info_path = 'S体系/20240611/S-体系-all metrics/shap_results-n-PrOH/MLP-features-effects.csv'
product_info_df = pd.read_csv(product_info_path, encoding='utf-8')

# 调试信息：检查列名
print("Features to search columns:", features_to_search_df.columns)
print("Product info columns:", product_info_df.columns)

# 预处理，确保特殊字符不会影响后续操作
features_to_search_df.columns = features_to_search_df.columns.str.replace('/', '_').str.replace(' ', '_')
product_info_df.columns = product_info_df.columns.str.replace('/', '_').str.replace(' ', '_')

# 调试信息：检查替换后的列名
print("Processed features to search columns:", features_to_search_df.columns)
print("Processed product info columns:", product_info_df.columns)

# 创建一个空的列表来存储所有的结果
results_list = []

# 处理每个 feature_to_search 并填入结果列表
for index, row in features_to_search_df.iterrows():
    feature_to_search = row[0]
    similar_features, explain_features = correlate_features(feature_to_search)

    product_info = product_info_df.loc[product_info_df['Feature'] == feature_to_search, 'Effect'].values
    product_value = product_info[0] if len(product_info) > 0 else ''

    # 确保所有条目都是包含两个值的元组
    similar_features = [item if isinstance(item, tuple) else ('None', 'None') for item in similar_features]
    
    max_len = max(len(similar_features), len(explain_features), 1)
#这里要改名字
    temp_df = pd.DataFrame({
        'Descriptor of interest': [feature_to_search] * max_len,
        'Effect on n-PrOH': [product_value] * max_len,
        'Similar Features': [f"{feature}, {correlation}" for feature, correlation in similar_features] + [''] * (max_len - len(similar_features)),
        'MFF': [f"{item[0]}, {item[1]}" if isinstance(item, tuple) else item for item in explain_features] + [''] * (max_len - len(explain_features))
    })

    results_list.append(temp_df)

results_df = pd.concat(results_list, ignore_index=True)

csv_save_dir = '20240611-S体系-data explanation'
os.makedirs(csv_save_dir, exist_ok=True)

csv_file_path = os.path.join(csv_save_dir, 'n-PrOH-features_analysis_results.csv')
results_df.to_csv(csv_file_path, index=False)

excel_file_path = os.path.join(csv_save_dir, 'n-PrOH-features_analysis_results.xlsx')
results_df.to_excel(excel_file_path, index=False, sheet_name='Analysis Results')

wb = load_workbook(excel_file_path)
ws = wb['Analysis Results']

current_feature = None
start_row = 2

for row in range(2, ws.max_row + 1):
    feature = ws[f'A{row}'].value
    if feature == current_feature:
        continue
    else:
        if current_feature is not None:
            ws.merge_cells(start_row=start_row, start_column=1, end_row=row-1, end_column=1)
            ws[f'A{start_row}'].alignment = Alignment(vertical='center')
        current_feature = feature
        start_row = row

if current_feature is not None:
    ws.merge_cells(start_row=start_row, start_column=1, end_row=ws.max_row, end_column=1)
    ws[f'A{start_row}'].alignment = Alignment(vertical='center')

current_feature = None
current_effect = None
start_row = 2

for row in range(2, ws.max_row + 1):
    effect = ws[f'B{row}'].value
    if effect == current_effect:
        continue
    else:
        if current_effect is not None:
            ws.merge_cells(start_row=start_row, start_column=2, end_row=row-1, end_column=2)
            ws[f'B{start_row}'].alignment = Alignment(vertical='center')
        current_effect = effect
        start_row = row

if current_effect is not None:
    ws.merge_cells(start_row=start_row, start_column=2, end_row=ws.max_row, end_column=2)
    ws[f'B{start_row}'].alignment = Alignment(vertical='center')

wb.save(excel_file_path)
print(f"Results saved to '{csv_file_path}' and '{excel_file_path}'")
print(results_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity

# 读取相关性系数矩阵的CSV文件
file_path = 'updated_correlation_matrix_between_Original_and_Encoded_Features.csv'
df = pd.read_csv(file_path, index_col=0)

# 数据预处理，去掉包含NaN或无穷大值的行
df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

# 转置数据，使每一行作为变量
df_transposed = df.transpose()

# 检查数据的适合性
kmo_all, kmo_model = calculate_kmo(df_transposed)
bartlett_chi_square, bartlett_p_value = calculate_bartlett_sphericity(df_transposed)

print(f"KMO Test: {kmo_model}")
print(f"Bartlett's Test: Chi-square={bartlett_chi_square}, p-value={bartlett_p_value}")

if bartlett_p_value < 0.05:
    print("The data is suitable for factor analysis.")
    
    # 因子分析
    Load_Matrix = FactorAnalyzer(n_factors=len(df_transposed.T), rotation=None, method='principal')
    Load_Matrix.fit(df_transposed)
    
    # 绘制碎石图以确定因子数量
    ev, v = Load_Matrix.get_eigenvalues()
    print('\n相关矩阵特征值：', ev)
    plt.figure(figsize=(8, 8))
    plt.scatter(range(1, df_transposed.shape[1] + 1), ev)
    plt.plot(range(1, df_transposed.shape[1] + 1), ev)
    plt.title('特征值和因子个数的变化', fontdict={'weight': 'normal', 'size': 25})
    plt.xlabel('Factors', fontdict={'weight': 'normal', 'size': 15})
    plt.ylabel('Eigenvalues', fontdict={'weight': 'normal', 'size': 15})
    plt.grid()
    plt.show()
    
    # 进行旋转后的因子分析
    Load_Matrix_rotated = FactorAnalyzer(rotation='varimax', n_factors=18, method='principal')
    Load_Matrix_rotated.fit(df_transposed)
    f_contribution_var_rotated = Load_Matrix_rotated.get_factor_variance()
    matrices_var_rotated = pd.DataFrame()
    matrices_var_rotated["特征值"] = f_contribution_var_rotated[0]
    matrices_var_rotated["方差贡献率"] = f_contribution_var_rotated[1]
    matrices_var_rotated["方差累计贡献率"] = f_contribution_var_rotated[2]
    print("旋转后的载荷矩阵的贡献率")
    print(matrices_var_rotated)
    print("旋转后的成分矩阵")
    print(Load_Matrix_rotated.loadings_)
    
    # 可视化因子载荷矩阵
    Load_Matrix = Load_Matrix_rotated.loadings_
    df_loadings = pd.DataFrame(np.abs(Load_Matrix), index=df_transposed.columns, columns=[f'Factor{i+1}' for i in range(18)])
    
    plt.rcParams['font.family'] = 'Times New Roman' # 设置英文字体为Times New Roman 
    plt.figure(figsize=(100, 100))
    ax = sns.heatmap(df_loadings, center=0, annot=True, cmap="coolwarm", cbar=False, fmt=".2f", annot_kws={"size": 30})
    ax.xaxis.set_tick_params(labelsize=40) # 设置x轴字体大小
    ax.yaxis.set_tick_params(labelsize=40) # 设置y轴字体大小
    plt.title('Factor Loadings Matrix', fontsize=40)
    plt.ylabel('Variables', fontsize=40)# 设置y轴标签
    plt.xlabel('Factors', fontsize=40)
    plt.show()# 显示图片
    
    # 计算因子得分
    factor_scores = Load_Matrix_rotated.transform(df_transposed)
    df_scores = pd.DataFrame(factor_scores, index=df_transposed.index, columns=[f'Factor{i+1}' for i in range(18)])
    
    # 打印和保存因子得分
    print("Factor Scores:")
    print(df_scores)
    
    scores_file_path = 'factor_scores.csv'
    df_scores.to_csv(scores_file_path)
    print(f"Factor scores saved to {scores_file_path}")
    # 可视化因子得分的热图
    plt.rcParams['font.family'] = 'Times New Roman'  # 设置英文字体为Times New Roman 
    plt.figure(figsize=(100, 200))
    ax = sns.heatmap(df_scores, annot=True, center=0, cmap="coolwarm", cbar=False, fmt=".2f", annot_kws={"size": 30})
    plt.title('Factor Scores of molecular physicochemical properties', fontsize=40)
    plt.xlabel('Factors', fontsize=40)
    plt.ylabel('Physicochemical properties', fontsize=40)
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=40) # 设置x轴字体大小
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=40) # 设置y轴字体大小
    plt.show()  # 显示图片
else:
    print("The data is not suitable for factor analysis.")




